## Setup and Imports

In [1]:
"""
=================================================================================
RNA Modification Comparison Notebook
=================================================================================
Purpose: Compare Direct RNA Sequencing (DRS) modification calls with orthogonal
         validation methods for HEK293 and GM12878 cell lines

Modifications analyzed:
- m6A (N6-methyladenosine) - GLORI-Seq 1.0 & 2.0+
- m5C (5-methylcytosine) - Published dataset
- Pseudouridine (Ψ) - BID-Seq & PRAISE
- Inosine (A-to-I editing) - SLIC-seq
- 2'-O-Methylation - Tang et al.

Author: [Your Name]
Date: [Current Date]
=================================================================================
"""

# Standard libraries
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from collections import defaultdict
from typing import Any, Union, Dict, Set
import gzip
import pickle
from functools import reduce

# Plotting libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm
from matplotlib_venn import venn2, venn3
import seaborn as sns

# Analysis libraries
from sklearn.metrics import mean_squared_error

# Configure matplotlib for publication-quality figures
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Helvetica']
mpl.rcParams['figure.dpi'] = 600
mpl.rcParams['savefig.dpi'] = 600
mpl.rcParams['savefig.transparent'] = True
mpl.rcParams['savefig.bbox'] = 'tight'
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

print("✓ All imports successful!")
print("✓ Matplotlib configured for publication-quality output")

✓ All imports successful!
✓ Matplotlib configured for publication-quality output


## Configuration

In [2]:
"""
=================================================================================
CONFIGURATION - UPDATE THESE PATHS FOR YOUR SYSTEM
=================================================================================
"""

# Base directory
BASE_DIR = Path("/Volumes/AJS_SSD")

# DRS data directories
HEK293_DRS_DIR = BASE_DIR / "HEK293" / "modkit_output" / "mod_specific_dataframes"
GM12878_DRS_DIR = BASE_DIR / "GM12878" / "mod_specific_dataframes"

# Orthogonal data paths
ORTHOG_DIR = BASE_DIR / "HEK293" / "orthogonal_datasets"

# m6A files
M6A_GLORI1_OLD = ORTHOG_DIR / "m6A" / "GLORI_1.0" / "41587_2022_1487_MOESM3_ESM.xlsx"
M6A_GLORI1_NEW = ORTHOG_DIR / "m6A" / "GLORI_1.0" / "41592_2025_2680_MOESM5_ESM(1).xlsb"
M6A_GLORI2_FILE = ORTHOG_DIR / "m6A" / "GLORI_2.0+" / "41592_2025_2680_MOESM3_ESM.xlsb"

# m5C file
M5C_FILE = ORTHOG_DIR / "m5C" / "GSE225614_HEK293T-WT_sites.tsv.gz"

# Pseudouridine files
PSI_BIDSEQ_FILE = ORTHOG_DIR / "psi" / "BID-Seq" / "GSE179798_HEK293T_mRNA_WT_BID-seq.xlsx"
PSI_PRAISE_FILE = ORTHOG_DIR / "psi" / "PRAISE" / "41589_2015_BFnchembio1836_MOESM158_ESM.xlsx"

# Inosine file
INO_FILE = ORTHOG_DIR / "inosine" / "Data_S2_A-to-I_sites_identified_by_slic-seq.xlsx"

# 2'OMe file
TWOME_FILE = ORTHOG_DIR / "2OMe" / "1-s2.0-S2667237524000365-mmc3.xlsx"

# GENCODE annotation
GENCODE_GTF = BASE_DIR / "HEK293" / "gencode_annotations" / "gencode.v47.annotation.gtf"

# Output directories
OUTPUT_DIR = BASE_DIR / "HEK293" / "scripts" / "notebooks" / "Plots" / "Plots_Updated_GLORI_Same_Sample"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

VALIDATED_DIR = BASE_DIR / "HEK293" / "orthogonal_validated"
VALIDATED_DIR.mkdir(parents=True, exist_ok=True)

print("✓ Configuration loaded")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  Validated sites directory: {VALIDATED_DIR}")

✓ Configuration loaded
  Output directory: /Volumes/AJS_SSD/HEK293/scripts/notebooks/Plots/Plots_Updated_GLORI_Same_Sample
  Validated sites directory: /Volumes/AJS_SSD/HEK293/orthogonal_validated


## Color Scheme Definition

In [3]:
"""
=================================================================================
COLOR SCHEME - Consistent colors for all plots
=================================================================================
"""

MODIFICATION_COLORS = {
    # m6A Technologies
    'm6a': {
        'DRS_HEK293': '#1f77b4',       # Blue
        'DRS_GM12878': '#aec7e8',      # Light blue
        'GLORI1': '#ff7f0e',           # Orange
        'GLORI2': '#d62728',           # Red
        'GLORI_combined': '#9467bd',   # Purple
        # Overlap colors for 3-way Venns
        'HEK_GLORI1': '#bcbd22',
        'HEK_GLORI2': '#17becf',
        'GLORI1_GLORI2': '#e377c2',
        'ALL_THREE': '#7f7f7f',
    },
    
    # m5C Technologies
    'm5c': {
        'DRS_HEK293': '#2ca02c',
        'DRS_GM12878': '#98df8a',
        'Orthogonal': '#e377c2',
        'HEK_Orth': '#8c564b',
        'GM_Orth': '#c49c94',
        'HEK_GM': '#bcbd22',
        'ALL_THREE': '#7f7f7f',
    },
    
    # Pseudouridine
    'psi': {
        'DRS_HEK293': '#9467bd',
        'DRS_GM12878': '#c5b0d5',
        'BID-seq': '#8c564b',
        'PRAISE': '#ff7f0e',
        'Combined': '#d62728',
        'HEK_BID': '#bcbd22',
        'HEK_PRAISE': '#17becf',
        'BID_PRAISE': '#e377c2',
        'ALL_THREE': '#7f7f7f',
    },
    
    # Inosine
    'inosine': {
        'DRS_HEK293': '#17becf',
        'DRS_GM12878': '#9edae5',
        'Orthogonal': '#ff7f0e',
        'HEK_Orth': '#8c564b',
        'GM_Orth': '#c49c94',
        'HEK_GM': '#bcbd22',
        'ALL_THREE': '#7f7f7f',
    },
    
    # 2'-O-Methylation
    '2ome': {
        'DRS_HEK293_A': '#e377c2',
        'DRS_HEK293_C': '#f7b6d2',
        'DRS_HEK293_G': '#7f7f7f',
        'DRS_HEK293_U': '#c7c7c7',
        'DRS_GM12878_A': '#1f77b4',
        'DRS_GM12878_C': '#aec7e8',
        'DRS_GM12878_G': '#ff7f0e',
        'DRS_GM12878_U': '#ffbb78',
        'Orthogonal_A': '#2ca02c',
        'Orthogonal_C': '#98df8a',
        'Orthogonal_G': '#d62728',
        'Orthogonal_U': '#ff9896',
    }
}

ALPHA = 0.7

HEATMAP_CMAPS = {
    'm6a_glori1': 'YlOrRd',
    'm6a_glori2': 'YlGnBu',
    'm6a_combined': 'RdPu',
    'm5c': 'Greens',
    'psi': 'Purples',
    'inosine': 'Blues',
    '2ome': 'Oranges',
}

print("✓ Color scheme loaded")

✓ Color scheme loaded


## Data Loading Class

In [4]:
"""
=================================================================================
UNIVERSAL DATA LOADER
=================================================================================
"""

class OrthogonalDataloader:
    """Universal dataloader for various file types"""
    def __init__(self, file_path: Union[Path, str]) -> None:
        self.file_path = Path(file_path)
        if not self.file_path.is_file():
            raise FileNotFoundError(f"File not found: '{self.file_path}'")

    def load_data(self, **kwargs: Any) -> Union[pd.DataFrame, Dict[str, pd.DataFrame], Any]:
        """Load data from file"""
        suffixes = self.file_path.suffixes
        compression = 'gzip' if '.gz' in suffixes else None
        
        print(f"  Loading '{self.file_path.name}'...")
        
        try:
            if '.pkl' in suffixes:
                if compression == 'gzip':
                    with gzip.open(self.file_path, 'rb') as f:
                        return pickle.load(f, **kwargs)
                else:
                    with open(self.file_path, 'rb') as f:
                        return pickle.load(f, **kwargs)
            
            elif '.xlsb' in suffixes or '.xlsx' in suffixes:
                if 'sheet_name' not in kwargs:
                    kwargs['sheet_name'] = None
                engine = 'pyxlsb' if '.xlsb' in suffixes else None
                return pd.read_excel(self.file_path, engine=engine, **kwargs)

            elif '.csv' in suffixes:
                return pd.read_csv(self.file_path, compression=compression, **kwargs)

            elif '.tsv' in suffixes or '.txt' in suffixes:
                if 'sep' not in kwargs:
                    kwargs['sep'] = '\t'
                return pd.read_csv(self.file_path, compression=compression, **kwargs)

            else:
                raise ValueError(f"Unsupported file type: {''.join(suffixes)}")
        
        except Exception as e:
            print(f"  ❌ Failed to load {self.file_path.name}: {e}")
            return None

print("✓ DataLoader class defined")

✓ DataLoader class defined


## Load DRS Data

In [5]:
"""
=================================================================================
LOAD DRS DATA (Dorado/Modkit Output)
=================================================================================
"""

def load_drs_data(base_dir: Path, cell_line: str) -> Dict[str, Any]:
    """Load all DRS modification data for a cell line"""
    mod_mapping = {
        'a': 'm6a',
        '17802': 'psi',
        '17596': 'inosine',
        'm': 'm5c',
        '19227': '2OMeU',
        '19228': '2OMeC',
        '19229': '2OMeG',
        '69426': '2OMeA'
    }
    
    drs_data = {}
    
    if not base_dir.exists():
        print(f"  ⚠️  Directory not found: {base_dir}")
        return drs_data
    
    parquet_files = [f for f in base_dir.iterdir() if f.suffix == '.parquet' and not f.name.startswith('.')]
    
    print(f"  Found {len(parquet_files)} files")
    
    for file in parquet_files:
        try:
            if 'filtered_' in file.name:
                mod_code = file.name.split('filtered_')[1].split('_')[0]
                mod_key = mod_mapping.get(mod_code, mod_code)
                
                df = pl.read_parquet(file)
                full_key = f"{cell_line}_{mod_key}"
                drs_data[full_key] = df
                print(f"    ✓ {full_key}: {len(df):,} rows")
        except Exception as e:
            print(f"    ❌ Failed {file.name}: {e}")
    
    return drs_data

print("\n" + "="*80)
print("LOADING DRS DATA")
print("="*80)

print("\n📂 Loading HEK293 DRS data...")
hek293_drs_data = load_drs_data(HEK293_DRS_DIR, 'HEK293')

print("\n📂 Loading GM12878 DRS data...")
gm12878_drs_data = load_drs_data(GM12878_DRS_DIR, 'GM12878')

# Combine
dorado_mods_dict = {**hek293_drs_data, **gm12878_drs_data}

print(f"\n✓ Loaded {len(dorado_mods_dict)} DRS datasets")
print(f"  Available: {list(dorado_mods_dict.keys())}")


LOADING DRS DATA

📂 Loading HEK293 DRS data...
  Found 8 files
    ✓ HEK293_m6a: 8,235,544 rows
    ✓ HEK293_psi: 8,312,485 rows
    ✓ HEK293_inosine: 8,235,544 rows
    ✓ HEK293_m5c: 6,732,738 rows
    ✓ HEK293_2OMeU: 8,312,485 rows
    ✓ HEK293_2OMeC: 6,732,738 rows
    ✓ HEK293_2OMeG: 7,104,902 rows
    ✓ HEK293_2OMeA: 8,235,544 rows

📂 Loading GM12878 DRS data...
  Found 8 files
    ✓ GM12878_m6a: 8,408,754 rows
    ✓ GM12878_psi: 8,122,125 rows
    ✓ GM12878_inosine: 8,408,754 rows
    ✓ GM12878_m5c: 7,161,645 rows
    ✓ GM12878_2OMeU: 8,122,125 rows
    ✓ GM12878_2OMeC: 7,161,645 rows
    ✓ GM12878_2OMeG: 7,575,765 rows
    ✓ GM12878_2OMeA: 8,408,754 rows

✓ Loaded 16 DRS datasets
  Available: ['HEK293_m6a', 'HEK293_psi', 'HEK293_inosine', 'HEK293_m5c', 'HEK293_2OMeU', 'HEK293_2OMeC', 'HEK293_2OMeG', 'HEK293_2OMeA', 'GM12878_m6a', 'GM12878_psi', 'GM12878_inosine', 'GM12878_m5c', 'GM12878_2OMeU', 'GM12878_2OMeC', 'GM12878_2OMeG', 'GM12878_2OMeA']


## Utility Functions

In [6]:
"""
=================================================================================
UTILITY FUNCTIONS
=================================================================================
"""

def get_drs_sites(mod_dict: Dict, cell_line: str, mod: str) -> Set[str]:
    """Extract DRS sites as set"""
    key = f"{cell_line}_{mod}"
    if key not in mod_dict:
        return set()
    
    df = mod_dict[key]
    if 'Adjusted_Mod_Proportion' in df.columns:
        df_filtered = df.filter((pl.col('Adjusted_Mod_Proportion') >= 20) & (pl.col('Score') >= 20))
    else:
        df_filtered = df.filter(pl.col('Score') >= 20)
    
    sample_chr = df_filtered['Chromosome'][0] if len(df_filtered) > 0 else None
    if sample_chr and not str(sample_chr).startswith('chr'):
        sites = set(('chr' + df_filtered['Chromosome'].cast(pl.Utf8) + '_' + 
                    df_filtered['End'].cast(pl.Int64).cast(pl.Utf8)).to_list())
    else:
        sites = set((df_filtered['Chromosome'].cast(pl.Utf8) + '_' + 
                    df_filtered['End'].cast(pl.Int64).cast(pl.Utf8)).to_list())
    return sites

def get_drs_values(mod_dict: Dict, cell_line: str, mod: str) -> pl.DataFrame:
    """Extract DRS sites WITH values"""
    key = f"{cell_line}_{mod}"
    if key not in mod_dict:
        return None
    
    df = mod_dict[key]
    if 'Adjusted_Mod_Proportion' in df.columns:
        df_filtered = df.filter((pl.col('Adjusted_Mod_Proportion') >= 20) & (pl.col('Score') >= 20))
    else:
        df_filtered = df.filter(pl.col('Score') >= 20)
    
    sample_chr = df_filtered['Chromosome'][0] if len(df_filtered) > 0 else None
    if sample_chr and not str(sample_chr).startswith('chr'):
        df_filtered = df_filtered.with_columns([
            ('chr' + pl.col('Chromosome').cast(pl.Utf8) + '_' + 
             pl.col('End').cast(pl.Int64).cast(pl.Utf8)).alias('site_id')
        ])
    else:
        df_filtered = df_filtered.with_columns([
            (pl.col('Chromosome').cast(pl.Utf8) + '_' + 
             pl.col('End').cast(pl.Int64).cast(pl.Utf8)).alias('site_id')
        ])
    return df_filtered.select(['site_id', 'Adjusted_Mod_Proportion'])

def process_orthogonal_sites(df, chr_col: str, pos_col: str) -> Set[str]:
    """Extract orthogonal sites as set"""
    if df is None:
        return set()
    
    if isinstance(df, pl.DataFrame):
        sample_chr = df[chr_col][0] if len(df) > 0 else None
        if sample_chr and not str(sample_chr).startswith('chr'):
            sites = set(('chr' + df[chr_col].cast(pl.Utf8) + '_' + 
                        df[pos_col].cast(pl.Float64).cast(pl.Int64).cast(pl.Utf8)).to_list())
        else:
            sites = set((df[chr_col].cast(pl.Utf8) + '_' + 
                        df[pos_col].cast(pl.Float64).cast(pl.Int64).cast(pl.Utf8)).to_list())
    else:
        sample_chr = str(df[chr_col].iloc[0]) if len(df) > 0 else None
        pos_int = df[pos_col].astype(float).astype(int).astype(str)
        if sample_chr and sample_chr.startswith('chr'):
            sites = set(df[chr_col].astype(str) + '_' + pos_int)
        else:
            sites = set('chr' + df[chr_col].astype(str) + '_' + pos_int)
    return sites

def process_orthogonal_values(df, chr_col: str, pos_col: str, value_col: str) -> pl.DataFrame:
    """Extract orthogonal sites WITH values"""
    if isinstance(df, pl.DataFrame):
        sample_chr = df[chr_col][0] if len(df) > 0 else None
        if sample_chr and not str(sample_chr).startswith('chr'):
            df_processed = df.with_columns([
                ('chr' + pl.col(chr_col).cast(pl.Utf8) + '_' + 
                 pl.col(pos_col).cast(pl.Float64).cast(pl.Int64).cast(pl.Utf8)).alias('site_id')
            ])
        else:
            df_processed = df.with_columns([
                (pl.col(chr_col).cast(pl.Utf8) + '_' + 
                 pl.col(pos_col).cast(pl.Float64).cast(pl.Int64).cast(pl.Utf8)).alias('site_id')
            ])
    else:
        ortho_clean = df.dropna(subset=[pos_col])
        sample_chr = str(ortho_clean[chr_col].iloc[0]) if len(ortho_clean) > 0 else None
        pos_int = ortho_clean[pos_col].astype(float).astype(int).astype(str)
        if sample_chr and sample_chr.startswith('chr'):
            site_ids = ortho_clean[chr_col].astype(str) + '_' + pos_int
        else:
            site_ids = 'chr' + ortho_clean[chr_col].astype(str) + '_' + pos_int
        df_processed = pl.DataFrame({
            'site_id': site_ids.tolist(),
            value_col: ortho_clean[value_col].tolist()
        })
    return df_processed.select(['site_id', value_col])

print("✓ Utility functions loaded")

✓ Utility functions loaded


In [7]:
"""
=================================================================================
VENN DIAGRAM HELPER FUNCTIONS
=================================================================================
"""

def plot_venn2_colored(sets, labels, colors, title, ax=None):
    """Plot 2-way Venn with custom colors"""
    if ax is None:
        fig, ax = plt.subplots(figsize=(4, 4))
    plt.sca(ax)
    venn2(sets, set_labels=labels, set_colors=colors, alpha=ALPHA)
    ax.set_title(title, fontweight='bold', fontsize=14)
    return ax

def plot_venn3_colored(sets, labels, patch_colors, title, ax=None):
    """Plot 3-way Venn with custom colors"""
    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
    plt.sca(ax)
    venn = venn3(sets, set_labels=labels)
    for region_id, color in patch_colors.items():
        patch = venn.get_patch_by_id(region_id)
        if patch:
            patch.set_facecolor(color)
            patch.set_alpha(ALPHA)
    ax.set_title(title, fontweight='bold', fontsize=14)
    return ax

print("✓ Venn helpers loaded")

✓ Venn helpers loaded


## Load GLORI Data

In [8]:
"""
=================================================================================
LOAD THREE GLORI DATASETS
=================================================================================
"""

print("\n" + "="*80)
print("LOADING THREE GLORI DATASETS")
print("="*80)

# ============================================================
# GLORI-1 NEW (3 replicates, 10ng sheet)
# ============================================================
print("\n📂 Loading GLORI-1 NEW (3 replicates)...")
loader = OrthogonalDataloader(M6A_GLORI1_NEW)
glori1_new_raw = loader.load_data()

if isinstance(glori1_new_raw, dict):
    print(f"  Multi-sheet Excel: {len(glori1_new_raw)} sheets")
    glori1_new_df = None
    for sheet_name, df in glori1_new_raw.items():
        if '10ng' in sheet_name:
            glori1_new_df = df
            print(f"  ✓ Using sheet: '{sheet_name}'")
            break
    if glori1_new_df is None:
        raise ValueError("Could not find GLORI-1 NEW sheet with '10ng' in name")
    glori1_new_raw = glori1_new_df
else:
    print(f"  Single DataFrame")

def process_glori1_new(df):
    """Process GLORI-1 NEW with 3 replicates (already in percentage)"""
    df = df.copy()
    if 'm6A_level_mean' in df.columns:
        return df
    # Data is already in percentage
    df['m6A_level_rep1_pct'] = df['m6A_level_rep1 (%)']
    df['m6A_level_rep2_pct'] = df['m6A_level_rep2 (%)']
    df['m6A_level_rep3_pct'] = df['m6A_level_rep3 (%)']
    df['m6A_level_mean'] = (df['m6A_level_rep1_pct'] + 
                            df['m6A_level_rep2_pct'] + 
                            df['m6A_level_rep3_pct']) / 3
    print(f"  ✓ Processed: {len(df):,} sites")
    print(f"     Mean m6A: {df['m6A_level_mean'].mean():.2f}%")
    print(f"     Range: {df['m6A_level_mean'].min():.2f}% - {df['m6A_level_mean'].max():.2f}%")
    return df

glori1_new = process_glori1_new(glori1_new_raw)

# ============================================================
# GLORI-1 OLD (2 replicates)
# ============================================================
print("\n📂 Loading GLORI-1 OLD (2 replicates)...")
loader = OrthogonalDataloader(M6A_GLORI1_OLD)  # Assuming different file path
glori1_old_raw = loader.load_data()

if isinstance(glori1_old_raw, dict):
    print(f"  Multi-sheet Excel: {len(glori1_old_raw)} sheets")
    glori1_old_df = None
    for sheet_name, df in glori1_old_raw.items():
        if 'm6A_level_rep1' in df.columns and 'm6A_level_rep2' in df.columns:
            glori1_old_df = df
            print(f"  ✓ Using sheet: '{sheet_name}'")
            break
    if glori1_old_df is None:
        raise ValueError("Could not find GLORI-1 OLD sheet with correct columns")
    glori1_old_raw = glori1_old_df
else:
    print(f"  Single DataFrame")

def process_glori1_old(df):
    """Process GLORI-1 OLD with 2 replicates"""
    df = df.copy()
    
    # Fix column name: "Sites" -> "Site" for consistency
    if 'Sites' in df.columns and 'Site' not in df.columns:
        df = df.rename(columns={'Sites': 'Site'})
        print(f"  ✓ Renamed 'Sites' -> 'Site' for consistency")
    
    if 'm6A_level_mean' in df.columns:
        return df
    # Convert to percentage
    df['m6A_level_rep1_pct'] = df['m6A_level_rep1'] * 100
    df['m6A_level_rep2_pct'] = df['m6A_level_rep2'] * 100
    df['m6A_level_mean'] = (df['m6A_level_rep1_pct'] + df['m6A_level_rep2_pct']) / 2
    if 'AGCov_rep1' in df.columns:
        df['AGCov_mean'] = (df['AGCov_rep1'] + df['AGCov_rep2']) / 2
    print(f"  ✓ Processed: {len(df):,} sites")
    print(f"     Mean m6A: {df['m6A_level_mean'].mean():.2f}%")
    print(f"     Range: {df['m6A_level_mean'].min():.2f}% - {df['m6A_level_mean'].max():.2f}%")
    return df

glori1_old = process_glori1_old(glori1_old_raw)

# ============================================================
# GLORI-2 (10ng)
# ============================================================
print("\n📂 Loading GLORI-2 (10ng)...")
loader = OrthogonalDataloader(M6A_GLORI2_FILE)
glori2_raw = loader.load_data()

def process_glori2_10ng(sheets_dict):
    """Load and process only the 10ng GLORI-2 data"""
    target_df = None
    for sheet_name, df in sheets_dict.items():
        if '10ng' in sheet_name and ('mRNA_input' in sheet_name or 'ng_mRNA' in sheet_name):
            target_df = df.copy()
            print(f"  ✓ Found sheet: '{sheet_name}'")
            break
    
    if target_df is None:
        raise ValueError("Could not find 10ng GLORI-2 sheet")
    
    # Find all m6A_level columns (replicates)
    m6a_cols = [col for col in target_df.columns if 'm6A_level' in col and col != 'm6A_level_mean']
    
    if m6a_cols:
        target_df['m6A_level_mean'] = target_df[m6a_cols].mean(axis=1)
        print(f"  ✓ Averaged {len(m6a_cols)} replicates")
    
    print(f"  ✓ Processed: {len(target_df):,} sites")
    print(f"     Mean m6A: {target_df['m6A_level_mean'].mean():.2f}%")
    print(f"     Range: {target_df['m6A_level_mean'].min():.2f}% - {target_df['m6A_level_mean'].max():.2f}%")
    return target_df

glori2 = process_glori2_10ng(glori2_raw)

print("\n✓ All three GLORI datasets loaded")
print(f"  GLORI-1 NEW: {len(glori1_new):,} sites")
print(f"  GLORI-1 OLD: {len(glori1_old):,} sites")
print(f"  GLORI-2:     {len(glori2):,} sites")


LOADING THREE GLORI DATASETS

📂 Loading GLORI-1 NEW (3 replicates)...
  Loading '41592_2025_2680_MOESM5_ESM(1).xlsb'...
  Multi-sheet Excel: 2 sheets
  ✓ Using sheet: '10ng_mRNA_input'
  ✓ Processed: 76,452 sites
     Mean m6A: 46.70%
     Range: 10.10% - 100.00%

📂 Loading GLORI-1 OLD (2 replicates)...
  Loading '41587_2022_1487_MOESM3_ESM.xlsx'...
  Multi-sheet Excel: 1 sheets
  ✓ Using sheet: 'Sheet1'
  ✓ Renamed 'Sites' -> 'Site' for consistency
  ✓ Processed: 170,240 sites
     Mean m6A: 45.52%
     Range: 10.00% - 100.00%

📂 Loading GLORI-2 (10ng)...
  Loading '41592_2025_2680_MOESM3_ESM.xlsb'...
  ✓ Found sheet: '10ng_mRNA_input'
  ✓ Averaged 3 replicates
  ✓ Processed: 101,613 sites
     Mean m6A: 45.40%
     Range: 10.14% - 100.00%

✓ All three GLORI datasets loaded
  GLORI-1 NEW: 76,452 sites
  GLORI-1 OLD: 170,240 sites
  GLORI-2:     101,613 sites


## Heatmap Functions

In [9]:
"""
=================================================================================
COMPLETE COMPARISON HEATMAPS (GLORI-to-GLORI + HEK293 DRS)
=================================================================================
"""

def plot_single_heatmap(ax, drs_df, ortho_df, drs_col, ortho_col, title, colormap):
    """Plot single 2D histogram heatmap"""
    merged = drs_df.join(ortho_df, on='site_id', how='inner')
    
    if len(merged) == 0:
        ax.text(0.5, 0.5, 'No overlapping sites', ha='center', va='center', fontsize=12)
        ax.set_title(title, fontweight='bold', fontsize=11)
        return
    
    drs_values = merged[drs_col].to_numpy()
    ortho_values = merged[ortho_col].to_numpy()
    
    bandwidth = 5
    bins = np.arange(0, 100 + bandwidth, bandwidth)
    hist, xedges, yedges = np.histogram2d(drs_values, ortho_values, bins=bins)
    
    im = ax.imshow(hist.T, norm=LogNorm(vmin=1, vmax=10**3),
                   origin='lower', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
                   aspect='auto', cmap=colormap, interpolation='nearest')
    
    cbar = plt.colorbar(im, ax=ax, ticks=np.logspace(0, 3, 4))
    cbar.set_ticklabels(['$10^0$', '$10^1$', '$10^2$', '$10^3$'])
    cbar.set_label('Site count', fontsize=10)
    
    correlation = np.corrcoef(drs_values, ortho_values)[0, 1]
    
    ax.set_xlabel('DRS Mod %', fontsize=11)
    ax.set_ylabel('Orthogonal Mod %', fontsize=11)
    ax.set_title(title, fontweight='bold', fontsize=12)
    ax.plot([0, 100], [0, 100], 'k--', linewidth=1.5)
    
    stats_text = f'n = {len(merged):,}\nr = {correlation:.3f}'
    ax.text(0.05, 0.95, stats_text, transform=ax.transAxes, 
            fontsize=10, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

def plot_complete_heatmaps(dorado_mods_dict, glori1_new, glori1_old, glori2, output_path=None):
    """Create heatmaps: GLORI-to-GLORI + HEK293 vs all GLORIs"""
    if output_path is None:
        output_path = OUTPUT_DIR / 'complete_comparison_heatmaps.pdf'
    
    print("\n" + "="*80)
    print("COMPLETE COMPARISON HEATMAPS")
    print("="*80)
    
    # Get HEK293 DRS data
    hek_drs = get_drs_values(dorado_mods_dict, 'HEK293', 'm6a')
    
    # Process all GLORI datasets
    glori1_new_df = process_orthogonal_values(glori1_new, 'Chr', 'Site', 'm6A_level_mean')
    glori1_old_df = process_orthogonal_values(glori1_old, 'Chr', 'Site', 'm6A_level_mean')
    glori2_df = process_orthogonal_values(glori2, 'Chr', 'Site', 'm6A_level_mean')
    
    fig = plt.figure(figsize=(18, 10))
    
    # Row 1: GLORI-to-GLORI comparisons
    ax1 = plt.subplot(2, 3, 1)
    plot_single_heatmap(ax1, glori1_new_df, glori1_old_df, 
                       'm6A_level_mean', 'm6A_level_mean',
                       'GLORI-1 NEW vs OLD', 'viridis')
    
    ax2 = plt.subplot(2, 3, 2)
    plot_single_heatmap(ax2, glori1_new_df, glori2_df,
                       'm6A_level_mean', 'm6A_level_mean',
                       'GLORI-1 NEW vs GLORI-2', 'plasma')
    
    ax3 = plt.subplot(2, 3, 3)
    plot_single_heatmap(ax3, glori1_old_df, glori2_df,
                       'm6A_level_mean', 'm6A_level_mean',
                       'GLORI-1 OLD vs GLORI-2', 'cividis')
    
    # Row 2: HEK293 DRS vs each GLORI
    ax4 = plt.subplot(2, 3, 4)
    plot_single_heatmap(ax4, hek_drs, glori1_new_df,
                       'Adjusted_Mod_Proportion', 'm6A_level_mean',
                       'HEK293 DRS vs GLORI-1 NEW', 'Blues')
    
    ax5 = plt.subplot(2, 3, 5)
    plot_single_heatmap(ax5, hek_drs, glori1_old_df,
                       'Adjusted_Mod_Proportion', 'm6A_level_mean',
                       'HEK293 DRS vs GLORI-1 OLD', 'Oranges')
    
    ax6 = plt.subplot(2, 3, 6)
    plot_single_heatmap(ax6, hek_drs, glori2_df,
                       'Adjusted_Mod_Proportion', 'm6A_level_mean',
                       'HEK293 DRS vs GLORI-2', 'Greens')
    
    fig.suptitle('Complete m6A Comparisons: GLORI Methods & HEK293 DRS', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(output_path, format='pdf', dpi=600, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved to {output_path}")

## Venn Diagram Functions

In [10]:
"""
=================================================================================
VENN DIAGRAM HELPER FUNCTIONS
=================================================================================
"""

def plot_venn2_colored(sets, labels, colors, title, ax=None):
    """Plot 2-way Venn with custom colors"""
    if ax is None:
        fig, ax = plt.subplots(figsize=(4, 4))
    plt.sca(ax)
    venn2(sets, set_labels=labels, set_colors=colors, alpha=ALPHA)
    ax.set_title(title, fontweight='bold', fontsize=14)
    return ax

def plot_venn3_colored(sets, labels, patch_colors, title, ax=None):
    """Plot 3-way Venn with custom colors"""
    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
    plt.sca(ax)
    venn = venn3(sets, set_labels=labels)
    for region_id, color in patch_colors.items():
        patch = venn.get_patch_by_id(region_id)
        if patch:
            patch.set_facecolor(color)
            patch.set_alpha(ALPHA)
    ax.set_title(title, fontweight='bold', fontsize=14)
    return ax

print("✓ Venn helpers loaded")

"""
=================================================================================
COMPLETE VENN DIAGRAMS (GLORI-to-GLORI + HEK293 DRS)
=================================================================================
"""



def plot_complete_venns(dorado_mods_dict, glori1_new, glori1_old, glori2, output_path=None):
    """Create Venn diagrams: GLORI-to-GLORI + HEK293 comparisons"""
    if output_path is None:
        output_path = OUTPUT_DIR / 'complete_comparison_venns.pdf'
    
    print("\n" + "="*80)
    print("COMPLETE VENN DIAGRAMS")
    print("="*80)
    
    # Extract site sets
    hek_sites = get_drs_sites(dorado_mods_dict, 'HEK293', 'm6a')
    glori1_new_sites = process_orthogonal_sites(glori1_new, 'Chr', 'Site')
    glori1_old_sites = process_orthogonal_sites(glori1_old, 'Chr', 'Site')
    glori2_sites = process_orthogonal_sites(glori2, 'Chr', 'Site')
    
    print(f"  HEK293 DRS:  {len(hek_sites):,} sites")
    print(f"  GLORI-1 NEW: {len(glori1_new_sites):,} sites")
    print(f"  GLORI-1 OLD: {len(glori1_old_sites):,} sites")
    print(f"  GLORI-2:     {len(glori2_sites):,} sites")
    
    # Define colors
    colors = {
        'hek': '#e74c3c',      # red
        'new': '#3498db',      # blue
        'old': '#f39c12',      # orange
        'glori2': '#2ecc71',   # green
    }
    
    fig = plt.figure(figsize=(18, 10))
    
    # Row 1: GLORI-only comparisons
    ax1 = plt.subplot(2, 3, 1)
    patch_colors_3way = {
        '100': colors['new'], '010': colors['old'], '001': colors['glori2'],
        '110': '#9b59b6', '101': '#1abc9c', '011': '#e67e22', '111': '#95a5a6'
    }
    plot_venn3_colored([glori1_new_sites, glori1_old_sites, glori2_sites],
                      ['GLORI-1 NEW', 'GLORI-1 OLD', 'GLORI-2'],
                      patch_colors_3way, 'All Three GLORI Methods', ax1)
    
    ax2 = plt.subplot(2, 3, 2)
    plot_venn2_colored([glori1_new_sites, glori1_old_sites],
                      ['GLORI-1 NEW', 'GLORI-1 OLD'],
                      [colors['new'], colors['old']],
                      'GLORI-1 Versions', ax2)
    
    ax3 = plt.subplot(2, 3, 3)
    glori1_union = glori1_new_sites | glori1_old_sites
    plot_venn2_colored([glori1_union, glori2_sites],
                      ['GLORI-1 (NEW ∪ OLD)', 'GLORI-2'],
                      ['#9b59b6', colors['glori2']],
                      'GLORI-1 vs GLORI-2', ax3)
    
    # Row 2: HEK293 vs each GLORI
    ax4 = plt.subplot(2, 3, 4)
    plot_venn2_colored([hek_sites, glori1_new_sites],
                      ['HEK293 DRS', 'GLORI-1 NEW'],
                      [colors['hek'], colors['new']],
                      'HEK293 vs GLORI-1 NEW', ax4)
    
    ax5 = plt.subplot(2, 3, 5)
    plot_venn2_colored([hek_sites, glori1_old_sites],
                      ['HEK293 DRS', 'GLORI-1 OLD'],
                      [colors['hek'], colors['old']],
                      'HEK293 vs GLORI-1 OLD', ax5)
    
    ax6 = plt.subplot(2, 3, 6)
    plot_venn2_colored([hek_sites, glori2_sites],
                      ['HEK293 DRS', 'GLORI-2'],
                      [colors['hek'], colors['glori2']],
                      'HEK293 vs GLORI-2', ax6)
    
    fig.suptitle('Complete m6A Site Overlaps: GLORI Methods & HEK293 DRS', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(output_path, format='pdf', dpi=600, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved to {output_path}")

✓ Venn helpers loaded


## 4Way Venn HEK293 DRS vs. GLORI

In [18]:
# Install if needed:
!pip install venn

"""
=================================================================================
FOUR-WAY VENN DIAGRAM
=================================================================================
"""

def plot_four_way_venn(dorado_mods_dict, glori1_new, glori1_old, glori2, output_path=None):
    """Create true 4-way Venn diagram"""
    from venn import venn
    
    if output_path is None:
        output_path = OUTPUT_DIR / 'four_way_venn_diagram.pdf'
    
    print("\n" + "="*80)
    print("FOUR-WAY VENN DIAGRAM")
    print("="*80)
    
    # Extract site sets
    hek_sites = get_drs_sites(dorado_mods_dict, 'HEK293', 'm6a')
    glori1_new_sites = process_orthogonal_sites(glori1_new, 'Chr', 'Site')
    glori1_old_sites = process_orthogonal_sites(glori1_old, 'Chr', 'Site')
    glori2_sites = process_orthogonal_sites(glori2, 'Chr', 'Site')
    
    print(f"  HEK293 DRS:  {len(hek_sites):,} sites")
    print(f"  GLORI-1 NEW: {len(glori1_new_sites):,} sites")
    print(f"  GLORI-1 OLD: {len(glori1_old_sites):,} sites")
    print(f"  GLORI-2:     {len(glori2_sites):,} sites")
    
    # Create 4-way Venn
    fig, ax = plt.subplots(figsize=(10, 10))
    
    venn_dict = {
        'HEK293 DRS': hek_sites,
        'GLORI-1 NEW': glori1_new_sites,
        'GLORI-1 OLD': glori1_old_sites,
        'GLORI-2': glori2_sites
    }
    
    venn(venn_dict, ax=ax, fontsize=10, legend_loc='upper right')
    ax.set_title('m6A Sites: Four-Way Comparison\nHEK293 DRS vs All GLORI Methods', 
                 fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(output_path, format='pdf', dpi=600, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved to {output_path}")
    
    # Print key statistics
    print("\n📊 Key Intersections:")
    all_four = hek_sites & glori1_new_sites & glori1_old_sites & glori2_sites
    all_glori = glori1_new_sites & glori1_old_sites & glori2_sites
    
    print(f"  All 4 methods:           {len(all_four):,} sites")
    print(f"  All 3 GLORI (no HEK):    {len(all_glori - hek_sites):,} sites")
    print(f"  HEK293 only:             {len(hek_sites - glori1_new_sites - glori1_old_sites - glori2_sites):,} sites")

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'venn' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'venn'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for venn: filename=venn-0.1.3-py3-none-any.whl size=19781 sha256=81b8ef15a1d5d4627830a2818ee9d76976e10ab3d2709bef4e04aabc929b0cc0
  Stored in directory: /Users/andrewstein/Library/Caches/pip/wheels/4b/ef/db/388ec89f2cea595e9130857edcc7a90f2fc5db5b2cbc59ce11
Successfully built venn


## UpSet Plot

In [16]:
# First install if needed:
!pip install upsetplot

"""
=================================================================================
UPSET PLOT: FOUR-WAY COMPARISON
=================================================================================
"""

def plot_upset_four_way(dorado_mods_dict, glori1_new, glori1_old, glori2, output_path=None):
    """Create UpSet plot showing all intersections of 4 datasets"""
    from upsetplot import plot as upsetplot
    from upsetplot import from_contents
    
    if output_path is None:
        output_path = OUTPUT_DIR / 'four_way_upset.pdf'
    
    print("\n" + "="*80)
    print("UPSET PLOT: FOUR-WAY COMPARISON")
    print("="*80)
    
    # Extract site sets
    hek_sites = get_drs_sites(dorado_mods_dict, 'HEK293', 'm6a')
    glori1_new_sites = process_orthogonal_sites(glori1_new, 'Chr', 'Site')
    glori1_old_sites = process_orthogonal_sites(glori1_old, 'Chr', 'Site')
    glori2_sites = process_orthogonal_sites(glori2, 'Chr', 'Site')
    
    print(f"  HEK293 DRS:  {len(hek_sites):,} sites")
    print(f"  GLORI-1 NEW: {len(glori1_new_sites):,} sites")
    print(f"  GLORI-1 OLD: {len(glori1_old_sites):,} sites")
    print(f"  GLORI-2:     {len(glori2_sites):,} sites")
    
    # Create UpSet data structure
    upset_data = from_contents({
        'HEK293 DRS': hek_sites,
        'GLORI-1 NEW': glori1_new_sites,
        'GLORI-1 OLD': glori1_old_sites,
        'GLORI-2': glori2_sites
    })
    
    # Create plot
    fig = plt.figure(figsize=(12, 8))
    upsetplot(upset_data, fig=fig, sort_by='cardinality', show_counts=True)
    fig.suptitle('m6A Sites: Four-Way Comparison', fontsize=16, fontweight='bold', y=0.98)
    
    plt.tight_layout()
    plt.savefig(output_path, format='pdf', dpi=600, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved to {output_path}")
    
    # Print key intersections
    print("\n📊 Key Intersections:")
    print(f"  All 4 methods:           {len(hek_sites & glori1_new_sites & glori1_old_sites & glori2_sites):,}")
    print(f"  All 3 GLORI only:        {len((glori1_new_sites & glori1_old_sites & glori2_sites) - hek_sites):,}")
    print(f"  HEK293 + all 3 GLORI:    {len(hek_sites & glori1_new_sites & glori1_old_sites & glori2_sites):,}")
    print(f"  HEK293 only:             {len(hek_sites - glori1_new_sites - glori1_old_sites - glori2_sites):,}")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for upsetplot: filename=upsetplot-0.9.0-py3-none-any.whl size=24866 sha256=e1270d251c834f272fb6e769f2631bdaef20c3a77c8e7e4d99e8205878492d0e
  Stored in directory: /Users/andrewstein/Library/Caches/pip/wheels/25/f7/e8/cf1a20f6721631b3607d04dfd5c31889cf5a512177fa2a0564
Successfully built upsetplot


## RMSE Functions

In [12]:
"""
=================================================================================
RMSE/RMSD CALCULATION FUNCTION
=================================================================================
"""

def calculate_rmse(drs_df, ortho_df, drs_col, ortho_col, 
                   chr_col_drs, pos_col_drs, chr_col_ortho, pos_col_ortho,
                   comparison_name, scale_ortho_by_100=False, show_examples=False,
                   ortho_already_processed=False):
    """Calculate RMSE - handles both pandas and polars"""
    
    print(f"\n{'─'*70}")
    print(f"{comparison_name}")
    print(f"{'─'*70}")
    
    # Determine if we're working with polars or pandas
    is_polars = isinstance(drs_df, pl.DataFrame) and isinstance(ortho_df, pl.DataFrame)
    
    if is_polars:
        # POLARS PATH
        if 'site_id' not in drs_df.columns:
            drs_processed = drs_df.with_columns([
                (pl.col(chr_col_drs).cast(pl.Utf8) + '_' + 
                 pl.col(pos_col_drs).cast(pl.Int64).cast(pl.Utf8)).alias('site_id')
            ]).select(['site_id', drs_col])
        else:
            drs_processed = drs_df.select(['site_id', drs_col])
            print(f"  ✓ Using pre-existing site_id in DRS")
        
        if not ortho_already_processed:
            sample_chr = ortho_df[chr_col_ortho][0] if len(ortho_df) > 0 else None
            if sample_chr and not str(sample_chr).startswith('chr'):
                ortho_processed = ortho_df.with_columns([
                    ('chr' + pl.col(chr_col_ortho).cast(pl.Utf8) + '_' + 
                     pl.col(pos_col_ortho).cast(pl.Float64).cast(pl.Int64).cast(pl.Utf8)).alias('site_id')
                ])
            else:
                ortho_processed = ortho_df.with_columns([
                    (pl.col(chr_col_ortho).cast(pl.Utf8) + '_' + 
                     pl.col(pos_col_ortho).cast(pl.Float64).cast(pl.Int64).cast(pl.Utf8)).alias('site_id')
                ])
            ortho_processed = ortho_processed.select(['site_id', ortho_col])
        else:
            ortho_processed = ortho_df
        
        merged = drs_processed.join(ortho_processed, on='site_id', how='inner')
        
    else:
        # PANDAS PATH
        drs_df_pd = drs_df.to_pandas() if isinstance(drs_df, pl.DataFrame) else drs_df
        ortho_df_pd = ortho_df.to_pandas() if isinstance(ortho_df, pl.DataFrame) else ortho_df
        
        # Process DRS
        if 'site_id' not in drs_df_pd.columns:
            drs_processed = drs_df_pd.copy()
            drs_processed['site_id'] = (
                drs_processed[chr_col_drs].astype(str) + '_' + 
                drs_processed[pos_col_drs].astype(int).astype(str)
            )
            drs_processed = drs_processed[['site_id', drs_col]]
        else:
            drs_processed = drs_df_pd[['site_id', drs_col]].copy()
            print(f"  ✓ Using pre-existing site_id in DRS")
        
        # Process Orthogonal
        if not ortho_already_processed:
            ortho_processed = ortho_df_pd.copy()
            sample_chr = str(ortho_processed[chr_col_ortho].iloc[0]) if len(ortho_processed) > 0 else None
            
            if sample_chr and not sample_chr.startswith('chr'):
                ortho_processed['site_id'] = (
                    'chr' + ortho_processed[chr_col_ortho].astype(str) + '_' + 
                    ortho_processed[pos_col_ortho].astype(float).astype(int).astype(str)
                )
            else:
                ortho_processed['site_id'] = (
                    ortho_processed[chr_col_ortho].astype(str) + '_' + 
                    ortho_processed[pos_col_ortho].astype(float).astype(int).astype(str)
                )
            ortho_processed = ortho_processed[['site_id', ortho_col]]
        else:
            ortho_processed = ortho_df_pd
        
        merged = pd.merge(drs_processed, ortho_processed, on='site_id', how='inner', suffixes=('_x', '_y'))
        
        # Handle column name conflicts
        drs_col_use = f'{drs_col}_x' if f'{drs_col}_x' in merged.columns else drs_col
        ortho_col_use = f'{ortho_col}_y' if f'{ortho_col}_y' in merged.columns else ortho_col
        
        # Convert to numpy
        drs_values = merged[drs_col_use].values
        ortho_values = merged[ortho_col_use].values
        
        print(f"  Matched: {len(merged):,}")
        if len(merged) == 0:
            return None
            
        valid_mask = ~(np.isnan(drs_values) | np.isnan(ortho_values))
        drs_values = drs_values[valid_mask]
        ortho_values = ortho_values[valid_mask]
        
        if len(drs_values) == 0:
            return None
        
        differences = drs_values - ortho_values
        rmse = np.sqrt(mean_squared_error(drs_values, ortho_values))
        mae = np.mean(np.abs(differences))
        correlation = np.corrcoef(drs_values, ortho_values)[0, 1]
        
        if show_examples:
            print(f"  Examples: ", end="")
            for i in range(min(3, len(drs_values))):
                print(f"({drs_values[i]:.1f},{ortho_values[i]:.1f}) ", end="")
            print()
        
        print(f"  RMSE: {rmse:.2f}%, MAE: {mae:.2f}%, r: {correlation:.3f}")
        
        return {
            'comparison': comparison_name,
            'n_sites': len(drs_values),
            'rmse': rmse,
            'rmsd': rmse,
            'mae': mae,
            'correlation': correlation,
            'r_squared': correlation**2,
            'mean_diff': np.mean(differences),
            'std_diff': np.std(differences),
            'drs_mean': drs_values.mean(),
            'ortho_mean': ortho_values.mean()
        }
    
    # POLARS PATH CONTINUED
    print(f"  Matched: {len(merged):,}")
    if len(merged) == 0:
        return None
    
    drs_values = merged[drs_col].to_numpy()
    ortho_values = merged[ortho_col].to_numpy()
    
    valid_mask = ~(np.isnan(drs_values) | np.isnan(ortho_values))
    drs_values = drs_values[valid_mask]
    ortho_values = ortho_values[valid_mask]
    
    if len(drs_values) == 0:
        return None
    
    differences = drs_values - ortho_values
    rmse = np.sqrt(mean_squared_error(drs_values, ortho_values))
    mae = np.mean(np.abs(differences))
    correlation = np.corrcoef(drs_values, ortho_values)[0, 1]
    
    if show_examples:
        print(f"  Examples: ", end="")
        for i in range(min(3, len(drs_values))):
            print(f"({drs_values[i]:.1f},{ortho_values[i]:.1f}) ", end="")
        print()
    
    print(f"  RMSE: {rmse:.2f}%, MAE: {mae:.2f}%, r: {correlation:.3f}")
    
    return {
        'comparison': comparison_name,
        'n_sites': len(drs_values),
        'rmse': rmse,
        'rmsd': rmse,
        'mae': mae,
        'correlation': correlation,
        'r_squared': correlation**2,
        'mean_diff': np.mean(differences),
        'std_diff': np.std(differences),
        'drs_mean': drs_values.mean(),
        'ortho_mean': ortho_values.mean()
    }

"""
=================================================================================
COMPLETE RMSE CALCULATIONS
=================================================================================
"""

def calculate_complete_rmse(dorado_mods_dict, glori1_new, glori1_old, glori2):
    """Calculate RMSE for all pairwise comparisons: GLORI-to-GLORI + HEK293"""
    
    print("\n" + "="*80)
    print("COMPLETE RMSE CALCULATIONS")
    print("="*80)
    
    # Get HEK293 DRS data
    hek_drs = get_drs_values(dorado_mods_dict, 'HEK293', 'm6a')
    
    results = []
    
    print("\n" + "─"*80)
    print("GLORI-TO-GLORI COMPARISONS")
    print("─"*80)
    
    # 1. GLORI-1 NEW vs OLD
    result1 = calculate_rmse(
        glori1_new, glori1_old,
        'm6A_level_mean', 'm6A_level_mean',
        'Chr', 'Site', 'Chr', 'Site',
        'GLORI-1 NEW vs OLD',
        scale_ortho_by_100=False,
        show_examples=True
    )
    if result1:
        results.append(result1)
    
    # 2. GLORI-1 NEW vs GLORI-2
    result2 = calculate_rmse(
        glori1_new, glori2,
        'm6A_level_mean', 'm6A_level_mean',
        'Chr', 'Site', 'Chr', 'Site',
        'GLORI-1 NEW vs GLORI-2',
        scale_ortho_by_100=False,
        show_examples=True
    )
    if result2:
        results.append(result2)
    
    # 3. GLORI-1 OLD vs GLORI-2
    result3 = calculate_rmse(
        glori1_old, glori2,
        'm6A_level_mean', 'm6A_level_mean',
        'Chr', 'Site', 'Chr', 'Site',
        'GLORI-1 OLD vs GLORI-2',
        scale_ortho_by_100=False,
        show_examples=True
    )
    if result3:
        results.append(result3)
    
    print("\n" + "─"*80)
    print("HEK293 DRS vs GLORI COMPARISONS")
    print("─"*80)
    
    # 4. HEK293 vs GLORI-1 NEW (hek_drs already has site_id)
    result4 = calculate_rmse(
        hek_drs, glori1_new,
        'Adjusted_Mod_Proportion', 'm6A_level_mean',
        'site_id', 'site_id', 'Chr', 'Site',  # Dummy chr/pos for drs since site_id exists
        'HEK293 DRS vs GLORI-1 NEW',
        scale_ortho_by_100=False,
        show_examples=True
    )
    if result4:
        results.append(result4)
    
    # 5. HEK293 vs GLORI-1 OLD
    result5 = calculate_rmse(
        hek_drs, glori1_old,
        'Adjusted_Mod_Proportion', 'm6A_level_mean',
        'site_id', 'site_id', 'Chr', 'Site',
        'HEK293 DRS vs GLORI-1 OLD',
        scale_ortho_by_100=False,
        show_examples=True
    )
    if result5:
        results.append(result5)
    
    # 6. HEK293 vs GLORI-2
    result6 = calculate_rmse(
        hek_drs, glori2,
        'Adjusted_Mod_Proportion', 'm6A_level_mean',
        'site_id', 'site_id', 'Chr', 'Site',
        'HEK293 DRS vs GLORI-2',
        scale_ortho_by_100=False,
        show_examples=True
    )
    if result6:
        results.append(result6)
    
    # Summary table
    if results:
        print("\n" + "="*80)
        print("COMPLETE SUMMARY")
        print("="*80)
        print(f"{'Comparison':<35} {'N Sites':>10} {'RMSE':>8} {'MAE':>8} {'r':>8}")
        print("-"*80)
        for r in results:
            print(f"{r['comparison']:<35} {r['n_sites']:>10,} {r['rmse']:>8.2f} "
                  f"{r['mae']:>8.2f} {r['correlation']:>8.3f}")
    
    return results

## Overlap Statistics

In [13]:
"""
=================================================================================
JACCARD INDEX AND OVERLAP STATISTICS
=================================================================================
"""

def calculate_jaccard_stats(set1, set2, name1, name2):
    """Calculate Jaccard index and related overlap statistics"""
    intersection = set1 & set2
    union = set1 | set2
    
    jaccard = len(intersection) / len(union) if len(union) > 0 else 0
    overlap_coef = len(intersection) / min(len(set1), len(set2)) if min(len(set1), len(set2)) > 0 else 0
    dice = 2 * len(intersection) / (len(set1) + len(set2)) if (len(set1) + len(set2)) > 0 else 0
    
    # Precision and recall (treating set1 as "truth" and set2 as "prediction")
    precision = len(intersection) / len(set2) if len(set2) > 0 else 0
    recall = len(intersection) / len(set1) if len(set1) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'comparison': f'{name1} vs {name2}',
        'set1_name': name1,
        'set2_name': name2,
        'set1_size': len(set1),
        'set2_size': len(set2),
        'intersection': len(intersection),
        'union': len(union),
        'jaccard_index': jaccard,
        'overlap_coefficient': overlap_coef,
        'dice_coefficient': dice,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'set1_only': len(set1 - set2),
        'set2_only': len(set2 - set1)
    }

def calculate_complete_jaccard(dorado_mods_dict, glori1_new, glori1_old, glori2):
    """Calculate Jaccard indices for all pairwise comparisons"""
    
    print("\n" + "="*80)
    print("JACCARD INDEX AND OVERLAP STATISTICS")
    print("="*80)
    
    # Extract site sets
    hek_sites = get_drs_sites(dorado_mods_dict, 'HEK293', 'm6a')
    glori1_new_sites = process_orthogonal_sites(glori1_new, 'Chr', 'Site')
    glori1_old_sites = process_orthogonal_sites(glori1_old, 'Chr', 'Site')
    glori2_sites = process_orthogonal_sites(glori2, 'Chr', 'Site')
    
    print(f"\nDataset sizes:")
    print(f"  HEK293 DRS:  {len(hek_sites):,} sites")
    print(f"  GLORI-1 NEW: {len(glori1_new_sites):,} sites")
    print(f"  GLORI-1 OLD: {len(glori1_old_sites):,} sites")
    print(f"  GLORI-2:     {len(glori2_sites):,} sites")
    
    results = []
    
    print("\n" + "─"*80)
    print("GLORI-TO-GLORI COMPARISONS")
    print("─"*80)
    
    # 1. GLORI-1 NEW vs OLD
    result1 = calculate_jaccard_stats(glori1_new_sites, glori1_old_sites, 
                                      'GLORI-1 NEW', 'GLORI-1 OLD')
    results.append(result1)
    print(f"\n{result1['comparison']}")
    print(f"  Intersection: {result1['intersection']:,} | Union: {result1['union']:,}")
    print(f"  Jaccard: {result1['jaccard_index']:.4f} | Dice: {result1['dice_coefficient']:.4f}")
    print(f"  Overlap coef: {result1['overlap_coefficient']:.4f}")
    
    # 2. GLORI-1 NEW vs GLORI-2
    result2 = calculate_jaccard_stats(glori1_new_sites, glori2_sites,
                                      'GLORI-1 NEW', 'GLORI-2')
    results.append(result2)
    print(f"\n{result2['comparison']}")
    print(f"  Intersection: {result2['intersection']:,} | Union: {result2['union']:,}")
    print(f"  Jaccard: {result2['jaccard_index']:.4f} | Dice: {result2['dice_coefficient']:.4f}")
    print(f"  Overlap coef: {result2['overlap_coefficient']:.4f}")
    
    # 3. GLORI-1 OLD vs GLORI-2
    result3 = calculate_jaccard_stats(glori1_old_sites, glori2_sites,
                                      'GLORI-1 OLD', 'GLORI-2')
    results.append(result3)
    print(f"\n{result3['comparison']}")
    print(f"  Intersection: {result3['intersection']:,} | Union: {result3['union']:,}")
    print(f"  Jaccard: {result3['jaccard_index']:.4f} | Dice: {result3['dice_coefficient']:.4f}")
    print(f"  Overlap coef: {result3['overlap_coefficient']:.4f}")
    
    print("\n" + "─"*80)
    print("HEK293 DRS vs GLORI COMPARISONS")
    print("─"*80)
    
    # 4. HEK293 vs GLORI-1 NEW
    result4 = calculate_jaccard_stats(hek_sites, glori1_new_sites,
                                      'HEK293 DRS', 'GLORI-1 NEW')
    results.append(result4)
    print(f"\n{result4['comparison']}")
    print(f"  Intersection: {result4['intersection']:,} | Union: {result4['union']:,}")
    print(f"  Jaccard: {result4['jaccard_index']:.4f} | Dice: {result4['dice_coefficient']:.4f}")
    print(f"  Overlap coef: {result4['overlap_coefficient']:.4f}")
    print(f"  Precision: {result4['precision']:.4f} | Recall: {result4['recall']:.4f} | F1: {result4['f1_score']:.4f}")
    
    # 5. HEK293 vs GLORI-1 OLD
    result5 = calculate_jaccard_stats(hek_sites, glori1_old_sites,
                                      'HEK293 DRS', 'GLORI-1 OLD')
    results.append(result5)
    print(f"\n{result5['comparison']}")
    print(f"  Intersection: {result5['intersection']:,} | Union: {result5['union']:,}")
    print(f"  Jaccard: {result5['jaccard_index']:.4f} | Dice: {result5['dice_coefficient']:.4f}")
    print(f"  Overlap coef: {result5['overlap_coefficient']:.4f}")
    print(f"  Precision: {result5['precision']:.4f} | Recall: {result5['recall']:.4f} | F1: {result5['f1_score']:.4f}")
    
    # 6. HEK293 vs GLORI-2
    result6 = calculate_jaccard_stats(hek_sites, glori2_sites,
                                      'HEK293 DRS', 'GLORI-2')
    results.append(result6)
    print(f"\n{result6['comparison']}")
    print(f"  Intersection: {result6['intersection']:,} | Union: {result6['union']:,}")
    print(f"  Jaccard: {result6['jaccard_index']:.4f} | Dice: {result6['dice_coefficient']:.4f}")
    print(f"  Overlap coef: {result6['overlap_coefficient']:.4f}")
    print(f"  Precision: {result6['precision']:.4f} | Recall: {result6['recall']:.4f} | F1: {result6['f1_score']:.4f}")
    
    # Additional: HEK293 vs Combined GLORI
    glori_union = glori1_new_sites | glori1_old_sites | glori2_sites
    glori_intersection = glori1_new_sites & glori1_old_sites & glori2_sites
    
    print("\n" + "─"*80)
    print("HEK293 vs COMBINED GLORI")
    print("─"*80)
    
    result7 = calculate_jaccard_stats(hek_sites, glori_union,
                                      'HEK293 DRS', 'GLORI (∪ all)')
    results.append(result7)
    print(f"\n{result7['comparison']}")
    print(f"  Intersection: {result7['intersection']:,} | Union: {result7['union']:,}")
    print(f"  Jaccard: {result7['jaccard_index']:.4f} | Dice: {result7['dice_coefficient']:.4f}")
    print(f"  Overlap coef: {result7['overlap_coefficient']:.4f}")
    
    result8 = calculate_jaccard_stats(hek_sites, glori_intersection,
                                      'HEK293 DRS', 'GLORI (∩ all)')
    results.append(result8)
    print(f"\n{result8['comparison']}")
    print(f"  Intersection: {result8['intersection']:,} | Union: {result8['union']:,}")
    print(f"  Jaccard: {result8['jaccard_index']:.4f} | Dice: {result8['dice_coefficient']:.4f}")
    print(f"  Overlap coef: {result8['overlap_coefficient']:.4f}")
    
    # Summary table
    print("\n" + "="*80)
    print("COMPLETE JACCARD SUMMARY")
    print("="*80)
    print(f"{'Comparison':<35} {'Jaccard':>8} {'Dice':>8} {'Overlap':>8} {'F1':>8}")
    print("-"*80)
    for r in results:
        f1_str = f"{r['f1_score']:.4f}" if r['f1_score'] > 0 else "N/A"
        print(f"{r['comparison']:<35} {r['jaccard_index']:>8.4f} {r['dice_coefficient']:>8.4f} "
              f"{r['overlap_coefficient']:>8.4f} {f1_str:>8}")
    
    return results

## Execution

In [14]:
"""
=================================================================================
RUN ALL COMPARISONS
=================================================================================
"""

# Generate all comparison plots and analyses
print("\n" + "="*80)
print("RUNNING COMPLETE ANALYSIS")
print("="*80)

plot_complete_heatmaps(dorado_mods_dict, glori1_new, glori1_old, glori2)
plot_complete_venns(dorado_mods_dict, glori1_new, glori1_old, glori2)
plot_four_way_comparison(dorado_mods_dict, glori1_new, glori1_old, glori2)
rmse_results = calculate_complete_rmse(dorado_mods_dict, glori1_new, glori1_old, glori2)
jaccard_results = calculate_complete_jaccard(dorado_mods_dict, glori1_new, glori1_old, glori2)

print("\n✓ All comparisons complete!")
print(f"  Total RMSE comparisons: {len(rmse_results)}")
print(f"  Total Jaccard comparisons: {len(jaccard_results)}")
print(f"  - GLORI-to-GLORI: 3")
print(f"  - HEK293 vs GLORI: 5")


RUNNING COMPLETE ANALYSIS

COMPLETE COMPARISON HEATMAPS


1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
/var/folders/dk/wjpkr6t136g7sz7r4x2c_ytr0000gn/T/ipykernel_3444/3192025944.py:98: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()


✓ Saved to /Volumes/AJS_SSD/HEK293/scripts/notebooks/Plots/Plots_Updated_GLORI_Same_Sample/complete_comparison_heatmaps.pdf

COMPLETE VENN DIAGRAMS
  HEK293 DRS:  67,517 sites
  GLORI-1 NEW: 76,452 sites
  GLORI-1 OLD: 170,240 sites
  GLORI-2:     101,613 sites


/var/folders/dk/wjpkr6t136g7sz7r4x2c_ytr0000gn/T/ipykernel_3444/3720266862.py:114: UserWarning: Glyph 8746 (\N{UNION}) missing from font(s) Helvetica.
  plt.tight_layout()
/var/folders/dk/wjpkr6t136g7sz7r4x2c_ytr0000gn/T/ipykernel_3444/3720266862.py:115: UserWarning: Glyph 8746 (\N{UNION}) missing from font(s) Helvetica.
  plt.savefig(output_path, format='pdf', dpi=600, bbox_inches='tight')
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
/var/folders/dk/wjpkr6t136g7sz7r4x2c_ytr0000gn/T/ipykernel_3444/3720266862.py:116: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/dk/wjpkr6t136g7sz7r4x2c_ytr0000gn/T/ipykernel_3444/563249674.py:54: UserWarning: Glyph 8746 (\N{UNION}) missing from font(s) Helvetica.
  plt.tight_layout()
/var/folders/dk/wjpkr6t136g7sz7r4x2c_ytr0000gn/T/ipykernel_3444/563249674.py:54: UserWarning: Glyph 8745 (\N{INTERSECTION}) missing from font(s) Helvetica.
  plt.t

✓ Saved to /Volumes/AJS_SSD/HEK293/scripts/notebooks/Plots/Plots_Updated_GLORI_Same_Sample/complete_comparison_venns.pdf

FOUR-WAY COMPARISON
  HEK293 DRS:              67,517 sites
  GLORI Union (all):       188,568 sites
  GLORI Intersection (∩3): 62,906 sites
✓ Saved to /Volumes/AJS_SSD/HEK293/scripts/notebooks/Plots/Plots_Updated_GLORI_Same_Sample/four_way_comparison.pdf

COMPLETE RMSE CALCULATIONS

────────────────────────────────────────────────────────────────────────────────
GLORI-TO-GLORI COMPARISONS
────────────────────────────────────────────────────────────────────────────────

──────────────────────────────────────────────────────────────────────
GLORI-1 NEW vs OLD
──────────────────────────────────────────────────────────────────────
  Matched: 67,231
  Examples: (64.1,52.9) (20.5,19.3) (23.3,26.9) 
  RMSE: 9.07%, MAE: 6.84%, r: 0.950

──────────────────────────────────────────────────────────────────────
GLORI-1 NEW vs GLORI-2
────────────────────────────────────────────

In [17]:
plot_upset_four_way(dorado_mods_dict, glori1_new, glori1_old, glori2)


UPSET PLOT: FOUR-WAY COMPARISON
  HEK293 DRS:  67,517 sites
  GLORI-1 NEW: 76,452 sites
  GLORI-1 OLD: 170,240 sites
  GLORI-2:     101,613 sites


/Users/andrewstein/miniforge3/envs/jupyter/lib/python3.13/site-packages/upsetplot/data.py:385: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(False, inplace=True)
/Users/andrewstein/miniforge3/envs/jupyter/lib/python3.13/site-packages/upsetplot/plotting.py:795: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.

✓ Saved to /Volumes/AJS_SSD/HEK293/scripts/notebooks/Plots/Plots_Updated_GLORI_Same_Sample/four_way_upset.pdf

📊 Key Intersections:
  All 4 methods:           29,913
  All 3 GLORI only:        32,993
  HEK293 + all 3 GLORI:    29,913
  HEK293 only:             15,530


/var/folders/dk/wjpkr6t136g7sz7r4x2c_ytr0000gn/T/ipykernel_3444/2456099982.py:48: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()


In [19]:
plot_four_way_venn(dorado_mods_dict, glori1_new, glori1_old, glori2)


FOUR-WAY VENN DIAGRAM
  HEK293 DRS:  67,517 sites
  GLORI-1 NEW: 76,452 sites
  GLORI-1 OLD: 170,240 sites
  GLORI-2:     101,613 sites


1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp


✓ Saved to /Volumes/AJS_SSD/HEK293/scripts/notebooks/Plots/Plots_Updated_GLORI_Same_Sample/four_way_venn_diagram.pdf

📊 Key Intersections:
  All 4 methods:           29,913 sites
  All 3 GLORI (no HEK):    32,993 sites
  HEK293 only:             15,530 sites


/var/folders/dk/wjpkr6t136g7sz7r4x2c_ytr0000gn/T/ipykernel_3444/2422153083.py:48: UserWarning: FigureCanvasPdf is non-interactive, and thus cannot be shown
  plt.show()
